In [3]:
from matplotlib import pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np

import sys

sys.path.insert(0, "../pompjax/pompjax/")

from ifeakf import ifeakf
from utils.samples2df import create_df_response
from diagnostic_plots.diagnostic_plot import convergence_plot

sys.path.insert(0, "../../")

from models import metapop_process_model

from global_config import config
import os

results_dir = config.get_property('results_dir')
model_name  = "mp model mean streams" #
#results_dir = os.path.join("..", "identifiability", model_name)
results_dir = os.path.join(results_dir, "identifiability", model_name)

os.makedirs(results_dir, exist_ok=True)

plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', titlesize=8)


ModuleNotFoundError: No module named 'metapopulation_models'